<a href="https://colab.research.google.com/github/znyi/svm-crf/blob/master/svm_crf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
#!rm -rf svm-crf

In [6]:
%cd /content/drive/My Drive/svm-crf
!git clone https://github.com/znyi/svm-crf.git

/content/drive/My Drive/svm-crf
fatal: destination path 'svm-crf' already exists and is not an empty directory.


In [7]:
#moved ipynb file to the cloned repo before %cd
%cd /content/drive/My Drive/svm-crf/svm-crf

/content/drive/My Drive/svm-crf/svm-crf


In [106]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [107]:
import numpy as np
from sklearn.svm import SVC
from keras.preprocessing.text import Tokenizer

xdata, ydata = [], []
with open('/content/drive/My Drive/svm-crf/svm-crf/SMSSpamCollection', 'r', encoding='utf8') as infile:
  lines = infile.readlines()
  for line in lines[:100]:
    part = line.strip().split('\t')
    label,sentence = part[0], part[1]
    ydata.append(label)
    xdata.append(sentence)

print("num of x data: ", len(xdata))
print("num of y data: ", len(ydata))

for i in range(10):
  print('x['+str(i)+'] - '+xdata[i])
  print('y['+str(i)+'] - '+ydata[i])

num of x data:  100
num of y data:  100
x[0] - Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
y[0] - ham
x[1] - Ok lar... Joking wif u oni...
y[1] - ham
x[2] - Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
y[2] - spam
x[3] - U dun say so early hor... U c already then say...
y[3] - ham
x[4] - Nah I don't think he goes to usf, he lives around here though
y[4] - ham
x[5] - FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, ￡1.50 to rcv
y[5] - spam
x[6] - Even my brother is not like to speak with me. They treat me like aids patent.
y[6] - ham
x[7] - As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
y[7] - ham
x[8] - WINNER!! A

In [108]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(xdata)
xidx,yidx = [], []
yidxdict = {'spam':1, 'ham':0}
xidx = tokenizer.texts_to_sequences(xdata)

maxlen = 60
for i in range(len(xidx)):
  length = len(xidx[i])
  if length>maxlen:
    xidx[i] = xidx[i][:maxlen]
  elif length<maxlen:
    xidx[i] = xidx[i] + [0]*(maxlen-length)
    
for label in ydata:
  yidx.append(yidxdict[label])

numtrain = int(0.9 * len(xidx))

xtrain = xidx[:numtrain]
ytrain = yidx[:numtrain]
xtest = xidx[numtrain:]
ytest = yidx[numtrain:]

print('size of train x:',len(xtrain))
print('size of train y:',len(ytrain))

size of train x: 90
size of train y: 90


In [109]:
svm = SVC(kernel='linear')
svm.fit(xtrain,ytrain)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [110]:
predict = svm.predict(xtest)
correct = 0
for i,p in enumerate(predict):
  if p == ytest[i]:
    correct += 1
accuracy = 100 * correct / len(ytest)

print('accuracy:',accuracy)

tosentence = tokenizer.sequences_to_texts(xtest)
tolabel = {1:'spam', 0:'ham'}

for i in range(len(predict)):
  print()
  print('sentence:',tosentence[i])
  print('correct :',tolabel[ytest[i]])
  print('predict :',tolabel[predict[i]])

accuracy: 80.0

sentence: yeah do don‘t stand to close tho you‘ll catch something
correct : ham
predict : spam

sentence: sorry to be a pain is it ok if we meet another night i spent late afternoon in casualty and that means i haven't done any of y stuff42moro and that includes all my time sheets and that sorry
correct : ham
predict : spam

sentence: smile in pleasure smile in pain smile when trouble pours like rain smile when sum1 hurts u smile becoz someone still loves to see u smiling
correct : ham
predict : ham

sentence: please call our customer service representative on 0800 169 6031 between 10am 9pm as you have won a guaranteed ￡1000 cash or ￡5000 prize
correct : spam
predict : spam

sentence: havent planning to buy later i check already lido only got 530 show in e afternoon u finish work already
correct : ham
predict : ham

sentence: your free ringtone is waiting to be collected simply text the password mix to 85069 to verify get usher and britney fml po box 5249 mk17 92h 450pp

In [111]:
!pip install sklearn_crfsuite

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [112]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
filepath = '/content/drive/My Drive/svm-crf/svm-crf/spacing_data.txt'

with open(filepath, 'r', encoding='utf8') as infile:
  lines = infile.readlines()

datas = []

for line in lines:
  part = line.strip().split('\t')
  seq = part[0].split()
  label = part[1].split()
  datas.append((seq, label))

numtrain = int(0.9*len(datas))
train_data = datas[:numtrain]
test_data = datas[numtrain:]

print(len(datas))
print(len(train_data))
print(len(test_data))
for data in datas[:5]:
  print(data)

1000
900
100
(['약', '속', '장', '소', '인', '신', '라', '호', '텔', '커', '피', '숍', '에', '재', '옥', '이', '먼', '저', '와', '기', '다', '리', '고', '있', '었', '다', '.'], ['B', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I'])
(['"', '야', '!', '나', '이', '든', '처', '녀', '가', '옷', '이', '라', '도', '좀', '화', '사', '한', '색', '으', '로', '입', '고', '다', '녀', '라', '.', '회', '색', '에', '다', '검', '정', '바', '바', '리', '가', '뭐', '니', '?', '"'], ['B', 'I', 'I', 'B', 'I', 'B', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'B', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I'])
(['재', '옥', '은', '전', '에', '없', '이', '정', '현', '의', '옷', '차', '림', '을', '탓', '하', '였', '다', '.'], ['B', 'I', 'I', 'B', 'I', 'B', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'I'])
(['"', '이', '게', '어', '때', '서', '?', '갑', '자', '기', '옷', '얘', '긴', '왜', '하', '니', '?', '선', '보', '일', '일', '있', 

In [78]:
def getfeat(seq):
  features = []

  for i in range(len(seq)):
    feature = {'BOF':True, 'EOF':True, 'WORD':seq[i], 'ISDIGIT':seq[i].isdigit()}
    if i == 0:
      feature['EOF'] = False
    elif i == len(seq)-1:
      feature['BOF'] = False
    if i-1>=0:
      feature['I-1'] = seq[i-1]
    if i-2>=0:
      feature['I-2'] = seq[i-2]
    if i+1<=len(seq)-1:
      feature['I+1'] = seq[i+1]
    if i+2<=len(seq)-1:
      feature['I+2'] = seq[i+2]
    features.append(feature)
  return features

train_x, train_y = [], []
test_x, test_y = [], []
train_size = int(0.9*len(datas))
for seqs, labs in train_data:
  train_x.append(getfeat(seqs))
  train_y.append(labs)
for seqs, labs in test_data:
  test_x.append(getfeat(seqs))
  test_y.append(labs)
for i in range(1):
  print(train_x[i])
  print(train_y[i])
  print(test_x[i])
  print(test_y[i])

[{'BOF': True, 'EOF': False, 'WORD': '약', 'ISDIGIT': False, 'I+1': '속', 'I+2': '장'}, {'BOF': True, 'EOF': True, 'WORD': '속', 'ISDIGIT': False, 'I-1': '약', 'I+1': '장', 'I+2': '소'}, {'BOF': True, 'EOF': True, 'WORD': '장', 'ISDIGIT': False, 'I-1': '속', 'I-2': '약', 'I+1': '소', 'I+2': '인'}, {'BOF': True, 'EOF': True, 'WORD': '소', 'ISDIGIT': False, 'I-1': '장', 'I-2': '속', 'I+1': '인', 'I+2': '신'}, {'BOF': True, 'EOF': True, 'WORD': '인', 'ISDIGIT': False, 'I-1': '소', 'I-2': '장', 'I+1': '신', 'I+2': '라'}, {'BOF': True, 'EOF': True, 'WORD': '신', 'ISDIGIT': False, 'I-1': '인', 'I-2': '소', 'I+1': '라', 'I+2': '호'}, {'BOF': True, 'EOF': True, 'WORD': '라', 'ISDIGIT': False, 'I-1': '신', 'I-2': '인', 'I+1': '호', 'I+2': '텔'}, {'BOF': True, 'EOF': True, 'WORD': '호', 'ISDIGIT': False, 'I-1': '라', 'I-2': '신', 'I+1': '텔', 'I+2': '커'}, {'BOF': True, 'EOF': True, 'WORD': '텔', 'ISDIGIT': False, 'I-1': '호', 'I-2': '라', 'I+1': '커', 'I+2': '피'}, {'BOF': True, 'EOF': True, 'WORD': '커', 'ISDIGIT': False, 'I-1': '텔', '

In [113]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs')
crf.fit(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [114]:
def print_result(test_data, predict):
  for i in range(len(test_data)):
    correct_sentence, predict_sentence = '', ''
    correct_seq, correct_lab = test_data[i]
    predict_lab = predict[i]
    for j in range(len(correct_seq)):
      if j == 0:
        correct_sentence += correct_seq[j]
        predict_sentence += correct_seq[j]
        continue
      if correct_lab[j] == 'B':
        correct_sentence += ' '
      correct_sentence += correct_seq[j] 
      if predict_lab[j] == 'B':
        predict_sentence += ' '
      predict_sentence += correct_seq[j]   
    print()    
    print('correct sentence:',correct_sentence)
    print('predict sentence:',predict_sentence)

predict = crf.predict(test_x)
accuracy = metrics.flat_accuracy_score(test_y,predict)*100
print('accuracy:',accuracy)
print_result(test_data[:5], predict[:5])

accuracy: 89.66043494849293

correct sentence: 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
predict sentence: 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분 모였다.

correct sentence: 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
predict sentence: 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

correct sentence: 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
predict sentence: 사라 예보에서 한 세르 비아인이 쏜총한 발이 합스부르크가의 계승 자를 죽였다.

correct sentence: 이 암살행위는 국지적인 민족주의들과 세계적인 제국주의들이 충돌하는 분쟁지역에서 저질러졌다.
predict sentence: 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

correct sentence: 오토만제국의 점진적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.
predict sentence: 오토만 제국의 점진 적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.


In [82]:
%cd /content/drive/My Drive/svm-crf/svm-crf

/content/drive/My Drive/svm-crf/svm-crf


In [100]:
!git init

Reinitialized existing Git repository in /content/drive/My Drive/svm-crf/svm-crf/.git/


In [102]:
!git add .

In [101]:
!git config --global user.email xinyi2709@gmail.com
!git config --global user.name znyi
!git commit -m "svm and crf"

[master 1105233] svm and crf
 1 file changed, 1 insertion(+), 1 deletion(-)


In [8]:
!git remote add origin https://github.com/znyi/svm-crf.git

fatal: remote origin already exists.


In [9]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address
